In [1]:
!pip install googlemaps scikit-learn pandas


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40748 sha256=f6929d1eb152b3689b2360db06799a90e6b295f320a1a34b97369f58dcd8ccc1
  Stored in directory: c:\users\ayush\appdata\local\pip\cache\wheels\4c\6a\a7\bbc6f5c200032025ee655deb5e163ce8594fa05e67d973aad6
Successfully built googlemaps


In [3]:

from transformers import AutoTokenizer, AutoModelForCausalLM

# Load GPT-J model and tokenizer from Hugging Face
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-j-6B")

# Generate response from GPT-J model
def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs['input_ids'], max_length=200)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response


tokenizer_config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

C:\Users\ayush\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ayush\.cache\huggingface\hub\models--EleutherAI--gpt-j-6B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.04k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

C:\Users\ayush\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/930 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/24.2G [00:00<?, ?B/s]

In [11]:
from transformers import MarianMTModel, MarianTokenizer
import torch

# English to Hindi translation model
en_to_hi_model = "Helsinki-NLP/opus-mt-en-hi"
tokenizer_hi = MarianTokenizer.from_pretrained(en_to_hi_model)
model_hi = MarianMTModel.from_pretrained(en_to_hi_model)

# Hindi to English translation model
hi_to_en_model = "Helsinki-NLP/opus-mt-hi-en"
tokenizer_en = MarianTokenizer.from_pretrained(hi_to_en_model)
model_en = MarianMTModel.from_pretrained(hi_to_en_model)
# Translate text from English to Hindi
def translate_to_hindi(text):
    inputs = tokenizer_hi.encode(text, return_tensors="pt")
    translated = model_hi.generate(inputs)
    return tokenizer_hi.decode(translated[0], skip_special_tokens=True)

# Translate text from Hindi to English
def translate_to_english(text):
    inputs = tokenizer_en.encode(text, return_tensors="pt")
    translated = model_en.generate(inputs)
    return tokenizer_en.decode(translated[0], skip_special_tokens=True)


In [13]:
# Define intents
intents = {
    "ticket_booking": ["book a ticket", "reserve a ticket", "buy tickets", "purchase tickets"],
    "museum_timings": ["museum timings", "when does the museum open", "opening hours", "closing hours"],
    "ticket_price": ["ticket cost", "ticket price", "how much are tickets", "cost of entry"],
    "museum_exhibitions": ["current exhibitions", "what exhibitions are on", "museum exhibits"],
    "museum_location": ["museum location", "where is the museum", "directions to the museum"],
    "places_nearby": ["places nearby", "nearby attractions", "famous places around"]
}

# Rule-based intent recognition function
def recognize_intent(user_input):
    user_input = user_input.lower()
    for intent, keywords in intents.items():
        for keyword in keywords:
            if keyword in user_input:
                return intent
    return "unknown_intent"

# Test the intent recognition
user_input = "Can I book 2 tickets?"
intent = recognize_intent(user_input)
print(f"Detected intent: {intent}")


Detected intent: unknown_intent


In [17]:
# Global state to track user input for ticket booking
user_state = {
    "intent": None,
    "tickets": None,
    "date": None
}

# Function to handle ticket booking conversation
def handle_ticket_booking(user_input):
    global user_state
    
    # Check for the number of tickets
    if user_state["tickets"] is None:
        user_state["tickets"] = input("How many tickets would you like to book?\n")
        return "How many tickets would you like to book?"
    
    # Check for the booking date
    elif user_state["date"] is None:
        user_state["date"] = input("Please provide the date for your booking.\n")
        return "Please provide the date for your booking."
    
    else:
        # Complete the booking process
        response = f"Booking confirmed for {user_state['tickets']} tickets on {user_state['date']}."
        user_state = {"intent": None, "tickets": None, "date": None}  # Reset the state after booking
        return response

# Example multi-turn conversation
user_input = "I want to book a ticket"
intent = recognize_intent(user_input)

if intent == "ticket_booking":
    response = handle_ticket_booking(user_input)
    print(response)


How many tickets would you like to book?
 hi


How many tickets would you like to book?


In [23]:
%pip install geopy

from geopy.geocoders import Nominatim

def get_museum_coordinates(museum_name):
    geolocator = Nominatim(user_agent="museum_chatbot")
    location = geolocator.geocode(museum_name)
    
    if location:
        return (location.latitude, location.longitude)
    else:
        return None

# Example to get the coordinates for National Museum, New Delhi
museum_name = "National Museum, New Delhi"
coords = get_museum_coordinates(museum_name)
if coords:
    lat, lon = coords
    print(f"Coordinates of {museum_name}: {lat}, {lon}")
else:
    print(f"Could not find coordinates for {museum_name}.")


Note: you may need to restart the kernel to use updated packages.
Coordinates of National Museum, New Delhi: 28.61191505, 77.2196390826266


In [25]:
import requests

def recommend_places_nearby_osm(lat, lon, radius=1000):
    overpass_url = "http://overpass-api.de/api/interpreter"
    
    # Query to get tourist attractions within a certain radius
    overpass_query = f"""
    [out:json];
    node
      ["tourism"="attraction"]
      (around:{radius},{lat},{lon});
    out body;
    """
    
    # Send request to Overpass API
    response = requests.get(overpass_url, params={'data': overpass_query})
    data = response.json()
    
    # Extract place names
    places = []
    for element in data['elements']:
        if 'name' in element:
            places.append(element['name'])
    
    return places

# Example to get places near the National Museum
if coords:
    nearby_places = recommend_places_nearby_osm(lat, lon)
    print(f"Famous places near {museum_name}: {', '.join(nearby_places)}")


Famous places near National Museum, New Delhi: 


In [32]:
def museum_chatbot(user_input, lat=None, lon=None):
    intent = recognize_intent(user_input)
    
    if intent == "ticket_booking":
        return handle_ticket_booking(user_input)
    
    elif intent == "museum_timings":
        return "The museum is open from 9 AM to 5 PM on weekdays."
    
    elif intent == "ticket_price":
        return "The ticket price is ₹200 for adults and ₹100 for children."
    
    elif intent == "museum_exhibitions":
        return "Currently, we have exhibitions on ancient Indian artifacts and sculptures."
    
    elif intent == "museum_location":
        return "The museum is located at Janpath, New Delhi."
    
    elif intent == "places_nearby" and lat and lon:
        places = nearby_places
        return f"Here are some famous places near the museum: {', '.join(places)}"
    
    else:
        return generate_response(f"User: {user_input}\nAssistant:")

# Test the complete chatbot
lat, lon = 28.6139, 77.2090  # Coordinates for National Museum, New Delhi
user_input = "Can you recommend places nearby the museum?"
response = museum_chatbot(user_input, lat, lon)
print(response)


Here are some famous places near the museum: 


In [34]:
def museum_chatbot(user_input, lat=None, lon=None):
    # Recognize the intent of the user
    intent = recognize_intent(user_input)
    
    # Handle ticket booking intent
    if intent == "ticket_booking":
        return handle_ticket_booking(user_input)
    
    # Handle museum timings intent
    elif intent == "museum_timings":
        return "The museum is open from 9 AM to 5 PM on weekdays."
    
    # Handle ticket price intent
    elif intent == "ticket_price":
        return "The ticket price is ₹200 for adults and ₹100 for children."
    
    # Handle museum exhibitions intent
    elif intent == "museum_exhibitions":
        return "Currently, we have exhibitions on ancient Indian artifacts and sculptures."
    
    # Handle museum location intent
    elif intent == "museum_location":
        return "The museum is located at Janpath, New Delhi."
    
    # Handle places nearby intent
    elif intent == "places_nearby" and lat and lon:
        places = recommend_places_nearby(lat, lon)
        return f"Here are some famous places near the museum: {', '.join(places)}"
    
    # Fallback to GPT-J for unknown intents
    else:
        return generate_response(f"User: {user_input}\nAssistant:")

# Coordinates for the National Museum in New Delhi (for testing location-based queries)
lat, lon = 28.6139, 77.2090

# Start the chatbot loop
print("Welcome to the Museum Chatbot! Type 'quit' to exit.")

while True:
    # Get user input
    user_input = input("You: ")
    
    # If the user types 'quit', exit the loop
    if user_input.lower() == 'quit':
        print("Goodbye!")
        break
    
    # Otherwise, handle the input with the chatbot
    response = museum_chatbot(user_input, lat, lon)
    print(f"Chatbot: {response}")


Welcome to the Museum Chatbot! Type 'quit' to exit.


You:  i


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


KeyboardInterrupt: 

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, MarianMTModel, MarianTokenizer
import requests
import torch

# Load GPT-J model and tokenizer
model_name = "EleutherAI/gpt-j-6B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Load translation model for multilingual support
translation_model_name = "Helsinki-NLP/opus-mt-en-ROMANCE"  # Example for English to Romance languages
translation_tokenizer = MarianTokenizer.from_pretrained(translation_model_name)
translation_model = MarianMTModel.from_pretrained(translation_model_name)

# Initialize conversation history
conversation_history = []

def translate(text, src_lang="en", tgt_lang="en"):
    if src_lang == tgt_lang:
        return text
    inputs = translation_tokenizer.encode(text, return_tensors="pt")
    outputs = translation_model.generate(inputs, forced_bos_token_id=translation_tokenizer.lang_code_to_id[tgt_lang])
    return translation_tokenizer.decode(outputs[0], skip_special_tokens=True)

def generate_response(prompt, history):
    # Add the prompt to the history
    history.append(prompt)
    
    # Prepare input for the model
    input_text = " ".join(history)
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=2048)
    
    # Generate the response
    output = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id,
        max_length=inputs['input_ids'].shape[1] + 50
    )
    
    # Decode the output
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    
    # Add the response to the history
    history.append(response)
    
    return response

def recommend_location(query):
    # Example using OpenStreetMap Nominatim API for location recommendations
    url = f"https://nominatim.openstreetmap.org/search?q={query}&format=json&addressdetails=1"
    response = requests.get(url)
    locations = response.json()
    
    if not locations:
        return "No locations found."
    
    # Return the top recommendation
    top_location = locations[0]
    return f"Recommended location: {top_location['display_name']}"

def museum_chatbot():
    print("Welcome to the Museum Chatbot! Type 'quit' to exit.")
    while True:
        user_input = input("You: ")
        
        if user_input.lower() == 'quit':
            print("Goodbye!")
            break
        
        # Detect language (assumed to be English for simplicity in this example)
        # You can use a language detection library like langdetect to detect language

        # Translate user input to English if needed
        user_input_en = translate(user_input, src_lang="es", tgt_lang="en")  # Example for Spanish to English
        
        # Check if the input is about location recommendation
        if "recommend" in user_input.lower():
            location_query = user_input  # Assume user input is a location query
            response = recommend_location(location_query)
        else:
            # Generate the response based on the current input and history
            response_en = generate_response(user_input_en, conversation_history)
            
            # Translate response back to the original language
            response = translate(response_en, src_lang="en", tgt_lang="es")  # Example for English to Spanish
        
        print(f"Chatbot: {response}")

# Start the chatbot
museum_chatbot()


C:\Users\ayush\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


KeyboardInterrupt: 